In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp forecast

# Forecast

> This module contains the forecast algorithms for a DGLM. These functions return a set of sampled values from the forecast distribution. Using this sample, PyBATS functions calculate point estimates (means, medians), and uncertainty measures such as credible intervals.

Forecasting occurs at time $t$ before a new value $y_t$ has been observed. There are two types of forecasts:

1. **Marginal** - A forecast made at *k* time steps into the future only. For example, marginal forecasting with $k=1$ will simulate from the forecast distribution for $y_t$. Marginal forecasting with $k=2$ will simulate from the forecast distribution for $y_{t+1}$, and so on.


2. **Path** - A joint forecast made from *1* to *k* steps into the future. Path forecasting with $k=3$ will simulate *dependent* observations $y_t, y_{t+1}, y_{t+2}$. Path forecasting is important when computing functions across multiple forecast horizons. The most common example is forecasting the *sum* $\sum_{h=1:k} y_{t+k-1}$. This is accomplished by path forecasting $k$ steps ahead, and then summing each path to get a random sample from the forecast distribution for the sum.

Naturally, forecasting $k-$steps ahead requires knowing all predictors $X$ that far into the future as well. Marginal forecasting $k-$steps ahead accepts $X_{t+k-1}$ as an argument. Path forecasting accepts the predictors in an array. Each row contains the predictors at one time step, starting with $X_t$ in the first row, down to $X_{t+k-1}$ in the last row.

In [ ]:
#hide
#exporti
import numpy as np
from scipy import stats
from scipy.special import gamma
from pybats.update import update_F

In [ ]:
#exporti
def forecast_aR(mod, k):
    Gk = np.linalg.matrix_power(mod.G, k - 1)
    a = Gk @ mod.a
    R = Gk @ mod.R @ Gk.T
    if mod.discount_forecast:
        R += (k - 1) * mod.W
    return a, R

In [ ]:
#exporti
def forecast_R_cov(mod, k1, k2):
    """
    :param mod: model
    :param k1: 1st Forecast Horizon (smaller)
    :param k2: 2nd Forecast Horizon (larger)
    :return: State vector covariance across k1, k2. West & Harrison the covariance is defined as Ct(k,j), pg. 106
    """
    if k2 < k1:
        tmp = k1
        k1 = k2
        k2 = tmp
    Gk = np.linalg.matrix_power(mod.G, k2 - k1)
    a, Rk1 = forecast_aR(mod, k1)
    return Gk @ Rk1

## Marginal Forecasting

In [ ]:
#export
def forecast_marginal(mod, k, X = None, nsamps = 1, mean_only = False, state_mean_var = False, y=None):
    """
    Marginal forecast function k steps ahead
    """
    # Plug in the correct F values
    F = update_F(mod, X, F=mod.F.copy())

    # Evolve to the prior for time t + k
    a, R = forecast_aR(mod, k)

    # Mean and variance
    ft, qt = mod.get_mean_and_var(F, a, R)

    if state_mean_var:
        return ft, qt

    # Choose conjugate prior, match mean and variance
    param1, param2 = mod.get_conjugate_params(ft, qt, mod.param1, mod.param2)

    if y is not None:
        return mod.loglik(y, param1, param2)

    if mean_only:
        return mod.get_mean(param1, param2)

    # Simulate from the forecast distribution
    return mod.simulate(param1, param2, nsamps)

This marginal forecast method works for Poisson and Bernoulli DGLMs. Below we run through a basic usage with the following steps:
- Manually define a simple DGLM with an intercept and 2 regression predictors
- Define the future predictors, $X_{t+k-1}$
- Forecast by drawing a random sample at $k=5$, and getting a $95\%$ credible interval
- Forecast the mean only, without drawing a random sample

In [ ]:
import numpy as np
from pybats.dglm import dlm, pois_dglm, bern_dglm, bin_dglm
from pybats.analysis import analysis

# Define Poisson and Bernoulli DGLMs, along with a normal DLM, with 1 trend term and 2 regression predictors
a0 = np.array([1, 1, 1])
R0 = np.eye(3)/10
mod_n = dlm(a0, R0, ntrend=1, nregn=2, deltrend=1, delregn=.9, discount_forecast=True)
mod_p = pois_dglm(a0, R0, ntrend=1, nregn=2, deltrend=1, delregn=.9)
mod_bern = bern_dglm(a0, R0, ntrend=1, nregn=2, deltrend=1, delregn=.9)

# New predictors for k=5 steps into the future
X = np.array([1,2])

First we're going to forecast from the Poisson DGLM, which will draw a random sample from the forecast distribution. We'll use the forecast samples to calculate a $95\%$ credible interval.

In [ ]:
# Forecast using the Poisson DGLM, i.e. simulate from the forecast distribution
forecast_samples = mod_p.forecast_marginal(k = 5, X=X, nsamps=1000)

# Get the 95% credible intervals
np.percentile(forecast_samples, [2.5, 97.5])

array([  9., 193.])

We can repeat this analysis using the normal DLM:

In [ ]:
# Forecast using the normal DLM, i.e. simulate from the forecast distribution
forecast_samples = mod_n.forecast_marginal(k = 5, X=X, nsamps=1000)

# Get the 95% credible intervals
np.percentile(forecast_samples, [2.5, 97.5])

array([-15.38467317,  19.76701873])

Next we'll test the models by setting `mean_only=True`. This will return only the mean value, without simulation from the forecast distribution. It is much faster if the mean is the only value of interest.

Notice that we skipped simulating from the Bernoulli DGLM, because Bernoulli outcomes are only $0$ or $1$ anyway. The mean of the forecast distribution in a Bernoulli DGLM is equal to the probability of the outcome being $1$.

In [ ]:
# Test the Poisson DGLM forecast mean is correct
# Setting the flag mean_only=True returns the exact mean, without simulation

m_p = mod_p.forecast_marginal(k = 5, X=X, mean_only=True)
ans = [70.38455503971952]
assert (np.equal(np.round(ans[0], 5), np.round(m_p, 5)).all())

# Test the Bernoulli DGLM forecast mean is correct
# Setting the flag mean_only=True returns the exact mean, without simulation

m_bern = mod_bern.forecast_marginal(k=5, X=X, mean_only=True)
ans = [0.9771442828394032]
assert (np.equal(np.round(ans[0], 5), np.round(m_bern, 5)).all())

# Test the Bernoulli DGLM forecast mean is correct
# Setting the flag state_mean_var=True returns the exact mean and variance, without simulation

m_n, v = mod_n.forecast_marginal(k = 5, X=X, state_mean_var=True)
ans = [4., 1.82222222]
assert(np.equal(np.round(ans[0], 5), np.round(m_n, 5)).all())
assert(np.equal(np.round(ans[1], 5), np.round(v, 5)).all())

`dglm.forecast_marginal` has three special features beyond returning just a random sample from the forecast distribution:

1. Setting `mean_only = True` makes the function return the mean of the forecast distribution, instead of a random sample.
2. Setting `state_mean_and_var = True` makes the function return the mean and variance of the linear predictor, $\lambda_t = F_t^{'} \theta_t$.
3. Passing in a value for `y` makes the function return the *log* probability density of `y` under the forecast distribution, $p(y)$.

In [ ]:
#export
def forecast_marginal_bindglm(mod, n, k, X=None, nsamps=1, mean_only=False):
    """
    Marginal forecast function k steps ahead for a binomial DGLM
    """
    # Plug in the correct F values
    F = update_F(mod, X, F=mod.F.copy())
    # F = np.copy(mod.F)
    # if mod.nregn > 0:
    #     F[mod.iregn] = X.reshape(mod.nregn,1)

    # Evolve to the prior for time t + k
    a, R = forecast_aR(mod, k)

    # Mean and variance
    ft, qt = mod.get_mean_and_var(F, a, R)

    # Choose conjugate prior, match mean and variance
    param1, param2 = mod.get_conjugate_params(ft, qt, mod.param1, mod.param2)

    if mean_only:
        return mod.get_mean(n, param1, param2)

    # Simulate from the forecast distribution
    return mod.simulate(n, param1, param2, nsamps)

We'll repeat the same tests from above on a binomial DGLM by specifying both the predictors `X` and the number of trials `n`.

In [ ]:
mod_b = bin_dglm(a0, R0, ntrend=1, nregn=2, deltrend=1, delregn=.9)

# Test the Binomial DGLM
m_bin = mod_b.forecast_marginal(n = 10, k=5, X=X, mean_only=True)
ans = [9.771442828394033]
assert (np.equal(np.round(ans[0], 5), np.round(m_bin, 5)).all())

In [ ]:
#exporti
def forecast_state_mean_and_var(mod, k = 1, X = None):
    """
   Forecast function that returns the mean and variance of lambda = state vector * predictors
   """
    # Plug in the correct F values
    F = update_F(mod, X, F=mod.F.copy())
    # F = np.copy(mod.F)
    # if mod.nregn > 0:
    #     F[mod.iregn] = X.reshape(mod.nregn, 1)

    # Evolve to the prior for time t + k
    a, R = forecast_aR(mod, k)

    # Mean and variance
    ft, qt = mod.get_mean_and_var(F, a, R)

    return ft, qt

## Path forecasting

In [ ]:
#export
def forecast_path(mod, k, X = None, nsamps = 1):
    """
    Path forecast function k steps ahead using sequential simulation
    """

    samps = np.zeros([nsamps, k])

    F = np.copy(mod.F)

    for n in range(nsamps):
        param1 = mod.param1
        param2 = mod.param2

        a = np.copy(mod.a)
        R = np.copy(mod.R)

        for i in range(k):

            # Plug in the correct F values
            if mod.nregn > 0:
                F = update_F(mod, X[i,:], F=F)
            # if mod.nregn > 0:
            #     F[mod.iregn] = X[i,:].reshape(mod.nregn,1)

            # Get mean and variance
            ft, qt = mod.get_mean_and_var(F, a, R)

            # Choose conjugate prior, match mean and variance
            param1, param2 = mod.get_conjugate_params(ft, qt, param1, param2)

            # Simulate next observation
            samps[n, i] = mod.simulate(param1, param2, nsamps = 1)

            # Update based on that observation
            param1, param2, ft_star, qt_star = mod.update_conjugate_params(samps[n, i], param1, param2)

            # Kalman filter update on the state vector (using Linear Bayes approximation)
            m = a + R @ F * (ft_star - ft)/qt
            C = R - R @ F @ F.T @ R * (1 - qt_star/qt)/qt

            # Get priors a, R for the next time step
            a = mod.G @ m
            R = mod.G @ C @ mod.G.T
            R = (R + R.T)/2

            # Discount information
            if mod.discount_forecast:
                R = R + mod.W

    return samps

In [ ]:
#export
def forecast_path_copula(mod, k, X = None, nsamps = 1, t_dist=False, y=None, nu=9, return_cov=False):
    """
    Path forecast function k steps ahead using a copula model
    """

    lambda_mu = np.zeros([k])
    lambda_cov = np.zeros([k, k])

    F = np.copy(mod.F)

    Flist = [None for x in range(k)]
    Rlist = [None for x in range(k)]

    for i in range(k):

        # Evolve to the prior at time t + i + 1
        a, R = forecast_aR(mod, i+1)

        Rlist[i] = R

        # Plug in the correct F values
        if mod.nregn > 0:
            F = update_F(mod, X[i, :], F=F)
        # if mod.nregn > 0:
        #     F[mod.iregn] = X[i,:].reshape(mod.nregn,1)

        Flist[i] = np.copy(F)

        # Find lambda mean and var
        ft, qt = mod.get_mean_and_var(F, a, R)
        lambda_mu[i] = ft
        lambda_cov[i,i] = qt

        # Find covariances with previous lambda values
        for j in range(i):
            # Covariance matrix between the state vector at times j, i, i > j
            cov_ij = np.linalg.matrix_power(mod.G, i-j) @ Rlist[j]
            # Covariance between lambda at times j, i
            lambda_cov[j,i] = lambda_cov[i,j] = Flist[j].T @ cov_ij @ Flist[i]

    if return_cov:
        return lambda_cov

    if y is not None:
        return forecast_path_copula_density_MC(mod, y, lambda_mu, lambda_cov, t_dist, nu, nsamps)
    else:
        return forecast_path_copula_sim(mod, k, lambda_mu, lambda_cov, nsamps, t_dist, nu)

In [ ]:
#export
def forecast_path_dlm(mod, k, X=None, nsamps=1, approx=True):
    """
    Path forecast function k steps ahead for a DLM
    """

    if approx:

        mean = np.zeros([k])
        cov = np.zeros([k, k])

        F = np.copy(mod.F)

        Flist = [None for x in range(k)]
        Rlist = [None for x in range(k)]

        for i in range(k):

            # Evolve to the prior at time t + i + 1
            a, R = forecast_aR(mod, i + 1)

            Rlist[i] = R

            # Plug in the correct F values
            if mod.nregn > 0:
                F = update_F(mod, X[i, :], F=F)

            Flist[i] = np.copy(F)

            # Find lambda mean and var
            ft, qt = mod.get_mean_and_var(F, a, R)
            mean[i] = ft
            cov[i, i] = qt

            # Find covariances with previous lambda values
            for j in range(i):
                # Covariance matrix between the state vector at times j, i, i > j
                cov_ij = np.linalg.matrix_power(mod.G, i - j) @ Rlist[j]
                # Covariance between lambda at times j, i
                cov[j, i] = cov[i, j] = Flist[j].T @ cov_ij @ Flist[i]

        return multivariate_t(mean, cov, mod.n, nsamps)

    else:

        samps = np.zeros([nsamps, k])
        F = np.copy(mod.F)
        p = len(F)

        ## Initialize samples of the state vector and variance from the prior
        v = 1.0 / np.random.gamma(shape=mod.n / 2, scale=2 / (mod.n * mod.s[0]), size=nsamps)
        thetas = np.array(list(
            map(lambda var: np.random.multivariate_normal(mean=mod.a.reshape(-1), cov=var / mod.s * mod.R, size=1).T,
                v))).squeeze()

        for i in range(k):

            # Plug in the correct F values
            if mod.nregn > 0:
                F = update_F(mod, X[i, :], F=F)

            # mean
            ft = (thetas @ F).reshape(-1)

            # Simulate from the sampling model
            samps[:, i] = mod.simulate_from_sampling_model(ft, v, nsamps)

            # Evolve the state vector and variance for the next timestep
            if mod.discount_forecast:
                v = v * np.random.beta(mod.delVar * mod.n / 2, ((1 - mod.delVar) * mod.n) / 2, size=nsamps)
                thetas = np.array(list(
                    map(lambda theta, var: mod.G @ theta + np.random.multivariate_normal(mean=np.zeros(p),
                                                                                         cov=var / mod.s * mod.W,
                                                                                         size=1),
                        thetas, v))).squeeze()
            else:
                v = v
                thetas = (mod.G @ thetas.T).T

        return samps

There are two functions for path forecasting with DGLMs. `forecast_path_copula` is the default method, and is part of my new [research](https://arxiv.org/pdf/2007.04956.pdf). It dramatically speeds up path forecasting in DGLMs. The traditional `forecast_path` function is only used when `dglm.forecast_path` is called with the flag `copula=FALSE`.

Below we'll test path forecasting - with a copula, as is the default - for $k=2$ steps into the future. The Poisson model will have $2$ trend terms - an intercept and a slope - and 1 regression term. This test will confirm that marginal forecasting and path forecasting lead to identical forecast means for $k=1$ and $k=2$ steps ahead.

In [ ]:
a0 = np.array([1, 1, 1])
R0 = np.eye(3)/100
mod_p = pois_dglm(a0, R0, ntrend=2, nregn=1, deltrend=1, delregn=.9)

# The predictors for the next 2 time steps, X_t and X_(t+1), in a matrix with k=2 rows
X = np.array([[1], [0.5]])

# Test the Poisson DGLM by comparing the path and marginal forecast distributions
samps_copula = mod_p.forecast_path_copula(k = 2, X=X, nsamps=100000)
m_samp = samps_copula.mean(axis=0)
m_marg = np.array([mod_p.forecast_marginal(k=1, X = X[0], mean_only=True),
          mod_p.forecast_marginal(k=2, X=X[1], mean_only=True)]).reshape(-1)
assert (np.equal(np.round(m_samp - m_marg, 1), np.zeros(2)).all())

samps_path = mod_p.forecast_path(k = 2, X=X, nsamps=2000, copula=False)
m_samp = samps_path.mean(axis=0)
assert (np.equal(np.round(m_samp - m_marg, 0), np.zeros(2)).all())

`dglm.forecast_path` has two special features beyond returning just a random sample from the forecast distribution. These only work when using `forecast_path_copula`, and are not yet available for a normal DLM.

1. Passing in a value (a vector of length $k$) for `y` makes the function return the *log* probability density of `y` under the forecast distribution, $p(y)$.
2. Setting `return_cov = True` makes the function return the covariance in the copula distribution. This will be different than the covariance in the forecast distribution, which can be calculated directly from a random sample.

In [ ]:
#exporti
def forecast_path_copula_sim(mod, k, lambda_mu, lambda_cov, nsamps, t_dist = False, nu = 9):
    """
    lambda_mu: kx1 Mean vector for forecast mean over t+1:t+k
    lambda_cov: kxk Covariance matrix for the forecast over t+1:t+k
    """

    if t_dist:
        #nu = 8
        scale = lambda_cov * ((nu - 2) / nu)
        joint_samps = multivariate_t(lambda_mu, scale, nu, nsamps).T
        genlist = list(map(lambda f, q: stats.t(loc = f, scale = np.sqrt(q), df = nu),
                      lambda_mu, np.diag(scale)))
    else:
        # Simulate from a joint multivariate normal with lambda_mu, lambda_cov
        joint_samps = np.random.multivariate_normal(lambda_mu, lambda_cov, size=nsamps).T
        genlist = list(map(lambda f, q: stats.norm(f, np.sqrt(q)),
                      lambda_mu, np.diag(lambda_cov)))

    # Find the marginal conjugate parameters
    conj_params = list(map(lambda f, q: mod.get_conjugate_params(f, q, mod.param1, mod.param2),
                           lambda_mu, np.diag(lambda_cov)))

    # Use the marginal CDF of the joint distribution to convert our samples into uniform RVs
    unif_rvs = list(map(lambda gen, samps: gen.cdf(samps),
              genlist, joint_samps))

    # Use inverse-CDF along each margin to get implied PRIOR value (e.g. a gamma dist RV for a poisson sampling model)
    priorlist = list(map(lambda params, unif_rv: mod.prior_inverse_cdf(unif_rv, params[0], params[1]),
                    conj_params, unif_rvs))

    # Simulate from the sampling model (e.g. poisson)
    return np.array(list(map(lambda prior: mod.simulate_from_sampling_model(prior, nsamps),
            priorlist))).T

In [ ]:
#exporti
def forecast_path_copula_density_MC(mod, y, lambda_mu, lambda_cov, t_dist=False, nu = 9, nsamps = 500):
    """
    Find the density of a sequence of future observations y
    """
    not_missing = np.logical_not(np.isnan(y))
    y = y[not_missing]
    lambda_mu = lambda_mu[not_missing]
    lambda_cov = lambda_cov[np.ix_(not_missing, not_missing)]

    # Get the marginals of the multivariate distribution
    if t_dist:
        scale = lambda_cov * ((nu - 2) / nu)
        joint_samps = multivariate_t(lambda_mu, scale, nu, nsamps).T
        genlist = list(map(lambda f, q: stats.t(loc=f, scale=np.sqrt(q), df=nu),
                           lambda_mu, np.diag(scale)))
    else:
        joint_samps = np.random.multivariate_normal(lambda_mu, lambda_cov, size=nsamps).T
        genlist = list(map(lambda f, q: stats.norm(f, np.sqrt(q)),
                           lambda_mu, np.diag(lambda_cov)))

    # Find the marginal distribution conjugate parameters
    conj_params = list(map(lambda f, q: mod.get_conjugate_params(f, q, mod.param1, mod.param2),
                           lambda_mu, np.diag(lambda_cov)))

    # Use the marginal CDF of the joint distribution to convert our samples into uniform RVs
    unif_rvs = list(map(lambda gen, samps: gen.cdf(samps),
                       genlist, joint_samps))

    # Use inverse-CDF along each margin to get implied PRIOR value (e.g. a gamma dist RV for a poisson sampling model)
    priorlist = list(map(lambda params, cdf: mod.prior_inverse_cdf(cdf, params[0], params[1]),
                         conj_params, unif_rvs))

    # Get the density of the y values, using Monte Carlo integration (i.e. an average over the samples)
    density_list = list(map(lambda y, prior: mod.sampling_density(y, prior),
                            y, priorlist))

    # Get the product of the densities to get the path density (they are independent, conditional upon the prior value at each time t+k)
    path_density_list = list(map(lambda dens: np.exp(np.sum(np.log(dens))),
                                 zip(*density_list)))

    # Return their average, on the log scale
    return np.log(np.mean(path_density_list))

In [ ]:
#exporti
def forecast_joint_copula_sim(mod_list, lambda_mu, lambda_cov, nsamps, t_dist=False, nu=9):
    """
    Jointly forecast from multiple DGLMs which share a latent factor
    """

    if t_dist:
        # nu = 8
        scale = lambda_cov * ((nu - 2) / nu)
        joint_samps = multivariate_t(lambda_mu, scale, nu, nsamps).T
        genlist = list(map(lambda f, q: stats.t(loc=f, scale=np.sqrt(q), df=nu),
                           lambda_mu, np.diag(scale)))
    else:
        # Simulate from a joint multivariate normal with lambda_mu, lambda_cov
        joint_samps = np.random.multivariate_normal(lambda_mu, lambda_cov, size=nsamps).T
        genlist = list(map(lambda f, q: stats.norm(f, np.sqrt(q)),
                           lambda_mu, np.diag(lambda_cov)))

    # Find the marginal conjugate parameters
    conj_params = []
    for i, mod in enumerate(mod_list):
        conj_params.append(mod.get_conjugate_params(lambda_mu[i], lambda_cov[i, i], mod.param1, mod.param2))

    # Use the marginal CDF of the joint distribution to convert our samples into uniform RVs
    unif_rvs = list(map(lambda gen, samps: gen.cdf(samps),
                        genlist, joint_samps))

    # If any are numerically 1 or 0, fix them:
    def numeric_fix(u):
        u[u == 1] = 1 - 1E-5
        u[u == 0] = 1E-5
        return u
    unif_rvs = [numeric_fix(u) for u in unif_rvs]

    # Use inverse-CDF along each margin to get implied PRIOR value (e.g. a gamma dist RV for a poisson sampling model)
    priorlist = list(map(lambda mod, params, unif_rv: mod.prior_inverse_cdf(unif_rv, params[0], params[1]),
                         mod_list, conj_params, unif_rvs))

    # Simulate from the sampling model (e.g. poisson)
    return np.array(list(map(lambda mod, prior: mod.simulate_from_sampling_model(prior, nsamps),
                             mod_list, priorlist))).T

In [ ]:
#exporti
def forecast_joint_copula_density_MC(mod_list, y, lambda_mu, lambda_cov, t_dist=False, nu = 9, nsamps = 500):
    """
    lambda_mu: kx1 Mean vector for forecast mean over t+1:t+k
    lambda_cov: kxk Covariance matrix for the forecast over t+1:t+k
    """
    not_missing = np.logical_not(np.isnan(y))
    y = y[not_missing]
    lambda_mu = lambda_mu[not_missing]
    lambda_cov = lambda_cov[np.ix_(not_missing, not_missing)]

    # Get the marginals of the multivariate distribution
    if t_dist:
        scale = lambda_cov * ((nu - 2) / nu)
        joint_samps = multivariate_t(lambda_mu, scale, nu, nsamps).T
        genlist = list(map(lambda f, q: stats.t(loc=f, scale=np.sqrt(q), df=nu),
                           lambda_mu, np.diag(scale)))
    else:
        joint_samps = np.random.multivariate_normal(lambda_mu, lambda_cov, size=nsamps).T
        genlist = list(map(lambda f, q: stats.norm(f, np.sqrt(q)),
                           lambda_mu, np.diag(lambda_cov)))

    # Find the marginal distribution conjugate parameters
    conj_params = []
    for i, mod in enumerate(mod_list):
        conj_params.append(mod.get_conjugate_params(lambda_mu[i], lambda_cov[i, i], mod.param1, mod.param2))

    # Use the marginal CDF of the joint distribution to convert our samples into uniform RVs
    unif_rvs = list(map(lambda gen, samps: gen.cdf(samps),
                       genlist, joint_samps))

    # Use inverse-CDF along each margin to get implied PRIOR value (e.g. a gamma dist RV for a poisson sampling model)
    priorlist = list(map(lambda mod, params, cdf: mod.prior_inverse_cdf(cdf, params[0], params[1]),
                         mod_list, conj_params, unif_rvs))

    # Get the density of the y values, using Monte Carlo integration (i.e. an average over the samples)
    density_list = list(map(lambda mod, y, prior: mod.sampling_density(y, prior),
                            mod_list, y, priorlist))

    # Get the product of the densities to get the joint density (they are independent, conditional upon the prior value at each time t+k)
    joint_density_list = list(map(lambda dens: np.exp(np.sum(np.log(dens))),
                                 zip(*density_list)))

    # Return their average, on the log scale
    return np.log(np.mean(joint_density_list))

In [ ]:
#exporti
def multivariate_t(mean, scale, nu, nsamps):
    '''
    mean = mean
    scale = covariance matrix * ((nu-2)/nu)
    nu = degrees of freedom
    nsamps = # of samples to produce
    '''
    p = len(mean)
    g = np.tile(np.random.gamma(nu/2.,2./nu, nsamps), (p, 1)).T
    Z = np.random.multivariate_normal(np.zeros(p), scale, nsamps)
    return mean + Z/np.sqrt(g)

In [ ]:
#exporti
def multivariate_t_density(y, mean, scale, nu):
    '''
    y = vector of observations
    mean = mean
    scale = covariance matrix * ((nu-2)/nu)
    nu = degrees of freedom
    '''
    y = y.reshape(-1, 1)
    mean = mean.reshape(-1, 1)
    dim = len(y)
    if dim > 1:
        constant = gamma((nu + dim) / 2) / (gamma(nu / 2) * np.sqrt((np.pi * nu) ** dim * np.linalg.det(scale)))
        dens = (1. + ((y - mean).T @ np.linalg.inv(scale) @ (y - mean)) / nu) ** (-(nu + dim) / 2)
    else:
        constant = gamma((nu + dim) / 2) / (gamma(nu / 2) * np.sqrt((np.pi * nu) ** dim * scale))
        dens = (1. + ((y - mean))**2 / (nu * scale)) ** (-(nu + dim) / 2)

    return 1. * constant * dens

In [ ]:
#exporti
def forecast_marginal_density_MC(mod, k, X = None, nsamps = 1, y = None):
    """
    Returns the log forecast density of an observation y
    """
    # Plug in the correct F values
    F = update_F(mod, X, F=mod.F.copy())
    # F = np.copy(mod.F)
    # if mod.nregn > 0:
    #     F[mod.iregn] = X.reshape(mod.nregn, 1)

    # Evolve to the prior for time t + k
    a, R = forecast_aR(mod, k)

    # Mean and variance
    ft, qt = mod.get_mean_and_var(F, a, R)

    # Choose conjugate prior, match mean and variance
    param1, param2 = mod.get_conjugate_params(ft, qt, mod.param1, mod.param2)

    # Simulate from the conjugate prior
    prior_samps = mod.simulate_from_prior(param1, param2, nsamps)

    # Get the densities
    densities = mod.sampling_density(y, prior_samps)

    # Take a Monte Carlo average, and return the mean density, on the log scale
    return np.log(np.mean(densities))

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_dglm.ipynb.
Converted 01_update.ipynb.
Converted 02_forecast.ipynb.
Converted 03_define_models.ipynb.
Converted 04_seasonal.ipynb.
Converted 05_analysis.ipynb.
Converted 06_conjugates.ipynb.
Converted 07_point_forecast.ipynb.
Converted 08_loss_functions.ipynb.
Converted 09_plot.ipynb.
Converted 10_shared.ipynb.
Converted 11_dcmm.ipynb.
Converted 12_dbcm.ipynb.
Converted 13_latent_factor.ipynb.
Converted 14_latent_factor_fxns.ipynb.
Converted 15_dlmm.ipynb.
Converted index.ipynb.
